<h1> Submitted by Ishan Ojha , CS-21 , Assignment - 4</h1>

Forward propagation in RNN's

In [1]:
import numpy as np

In [2]:
import numpy as np
def softmax(x):
    e_x = np.exp(x - np.max(x))  
    return e_x / e_x.sum(axis=0)


In [3]:

#input sequence of indices
x = [0, 1, 2, 1, 0]  
input_dim = 3        # Size of input vocabulary
hidden_dim = 5       # Size of the hidden layer
output_dim = 3       # Size of the output 


In [4]:

#  Initializing weights
U = np.random.uniform(
    -np.sqrt(1.0 / input_dim), np.sqrt(1.0 / input_dim),
    (hidden_dim, input_dim)
)
W = np.random.uniform(
    -np.sqrt(1.0 / hidden_dim), np.sqrt(1.0 / hidden_dim),
    (hidden_dim, hidden_dim)
)
V = np.random.uniform(
    -np.sqrt(1.0 / hidden_dim), np.sqrt(1.0 / hidden_dim),
    (output_dim, hidden_dim)
)


In [5]:
#no of steps
num_time_steps = len(x)


In [6]:

#hidden states
hidden_state = np.zeros((num_time_steps + 1, hidden_dim))
hidden_state[-1] = np.zeros(hidden_dim)


In [7]:

#initializing output
YHat = np.zeros((num_time_steps, output_dim))


In [8]:

#forward pass 
for t in np.arange(num_time_steps):
    # Compute hidden state
    # U[:, x[t]] selects the column of U corresponding to the input index x[t]
    hidden_state[t] = np.tanh(U[:, x[t]] + W.dot(hidden_state[t - 1]))


    YHat[t] = softmax(V.dot(hidden_state[t]))


print("Predicted outputs (YHat):")
print(YHat)


Predicted outputs (YHat):
[[0.32998354 0.31817976 0.3518367 ]
 [0.33779166 0.34470571 0.31750263]
 [0.36640489 0.3560724  0.27752271]
 [0.35577164 0.35175996 0.2924684 ]
 [0.33885873 0.31881058 0.34233068]]


Generating song lyrics using RNN's

In [9]:
import warnings
warnings.filterwarnings('ignore')

import random
import numpy as np
import pandas as pd
import tensorflow as tf

import logging
logging.getLogger('tensorflow').setLevel(logging.INFO)

In [10]:
#importing song data
df = pd.read_csv('songdata.csv')


In [11]:
df.head(

)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [12]:
#no of lyrics in dataset
df.shape[0]

57650

In [13]:
#no of song lyrics
len(df['artist'].unique())

643

In [14]:
    # no of songs from each artist
count =df['artist'].value_counts()[:10]
print(count)
print("\n")
df['artist'].value_counts().values.mean()

artist
Donna Summer        191
Gordon Lightfoot    189
Bob Dylan           188
George Strait       188
Loretta Lynn        187
Cher                187
Alabama             187
Reba Mcentire       187
Chaka Khan          186
Dean Martin         186
Name: count, dtype: int64




89.65785381026438

In [15]:
#combining all row of that column as data as the text variable 
data = ', '.join(df['text'])
data[:369]

"Look at her face, it's a wonderful face  \nAnd it means something special to me  \nLook at the way that she smiles when she sees me  \nHow lucky can one fellow be?  \n  \nShe's just my kind of girl, she makes me feel fine  \nWho could ever believe that she could be mine?  \nShe's just my kind of girl, without her I'm blue  \nAnd if she ever leaves me what could I do, what co"

In [16]:
chars = sorted(list(set(data)))
vocab_size = len(chars)
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}
char_to_ix['s']
ix_to_char[68]

's'

In [17]:
vocabSize = 7
char_index = 4

np.eye(vocabSize)[char_index]

array([0., 0., 0., 0., 1., 0., 0.])

In [18]:
#return the one hot encoded vectors

def one_hot_encoder(index):
    return np.eye(vocab_size)[index]

Defining network parameters

In [19]:
from tensorflow.keras import Input

In [20]:
#define the number of units in the hidden layer:
hidden_size = 100  
 
#define the length of the input and output sequence:
seq_length = 25  

#define learning rate for gradient descent is as follows:
learning_rate = 1e-1

#set the seed value:
seed_value = 42
# Set seeds for reproducibility
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)

In [21]:
inputs = Input(shape=[None, vocab_size],dtype=tf.float32, name="inputs")
targets = Input(shape=[None, vocab_size], dtype=tf.float32, name="targets")
init_state =Input(shape=[1, hidden_size], dtype=tf.float32, name="state")
initializer = tf.random_normal_initializer(stddev=0.1)

In [22]:
import tensorflow as tf

# Hyperparameters
vocab_size = 1000
hidden_size = 128
seq_length = 10
initializer = tf.keras.initializers.GlorotUniform()

# Simulate input: (seq_length, vocab_size)
inputs = tf.random.uniform((seq_length, vocab_size))
init_state = tf.zeros((hidden_size,))

# Define parameters
U = tf.Variable(initializer(shape=(vocab_size, hidden_size)), name="U")
W = tf.Variable(initializer(shape=(hidden_size, hidden_size)), name="W")
V = tf.Variable(initializer(shape=(hidden_size, vocab_size)), name="V")
bh = tf.Variable(tf.zeros([hidden_size]), name="bh")
by = tf.Variable(tf.zeros([vocab_size]), name="by")

# Forward pass
h_t = init_state
y_hat = []

for t in range(seq_length):
    x_t = inputs[t]  # Shape: (vocab_size,)
    h_t = tf.tanh(tf.matmul(tf.expand_dims(x_t, 0), U) + tf.matmul(tf.expand_dims(h_t, 0), W) + bh)
    y_t = tf.matmul(h_t, V) + by
    y_hat.append(tf.squeeze(y_t))  # Remove batch dimension for consistency

# Stack outputs: shape (seq_length, vocab_size)
y_hat = tf.stack(y_hat, axis=0)

print("Output shape:", y_hat.shape)  # Should be (seq_length, vocab_size)


Output shape: (10, 1000)


In [23]:
output_softmax = tf.nn.softmax(y_hat[-1])  

outputs = tf.concat(y_hat, axis=0)

In [24]:
class CrossEntropyLayer(tf.keras.layers.Layer):
    def call(self, logits, labels):
        return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits))

loss_layer = CrossEntropyLayer()
loss = loss_layer(outputs, targets)

In [25]:
hprev = h_t

In [26]:
optimizer = tf.keras.optimizers.Adam()
loss_layer = CrossEntropyLayer()

with tf.GradientTape() as tape:
    h_t = init_state
    y_hat = []

    for t in range(seq_length):
        x_t = inputs[t]
        h_t = tf.tanh(tf.matmul(tf.expand_dims(x_t, 0), U) +
                      tf.matmul(tf.expand_dims(h_t, 0), W) + bh)
        y_t = tf.matmul(h_t, V) + by
        y_hat.append(tf.squeeze(y_t))

    # Stack outputs: (seq_length, vocab_size)
    outputs = tf.stack(y_hat, axis=0)

    # Create one-hot encoded targets for use with softmax_cross_entropy_with_logits
    target_indices = tf.random.uniform((seq_length,), minval=0, maxval=vocab_size, dtype=tf.int32)
    targets = tf.one_hot(target_indices, depth=vocab_size)

    # Compute loss using your custom layer
    loss = loss_layer(outputs, targets)

# Clip gradients and apply
model_variables = [U, W, V, bh, by]
gradients = tape.gradient(loss, model_variables)
clipped_gradients = [tf.clip_by_value(g, -5.0, 5.0) for g in gradients]
optimizer.apply_gradients(zip(clipped_gradients, model_variables))

print("Loss:", loss.numpy())


Loss: 6.953747


In [27]:
print("Eager execution is enabled:", tf.executing_eagerly())


Eager execution is enabled: True
